<a href="https://colab.research.google.com/github/MrWou/Calcolo-Matrici-e-Distanze-Comuni-Emilia-Romagna/blob/main/Calcolo_matrici_e_distanze_Comuni_ER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installa Open Route Service

In [ ]:
!pip install --upgrade openrouteservice

# Import localities from ERT

Questo script Python utilizza l'API di Emilia Romagna Turismo (ERT) per estrarre un elenco di località.

Viene effettuata una richiesta paginata all'endpoint `/opendata/v1/localities` dell'API per recuperare tutte le località disponibili, con un limite di 100 record per pagina.

I dati estratti vengono poi elaborati e organizzati in un DataFrame Pandas (`df_comuni`) contenente le seguenti informazioni per ciascuna località:

- `id`: ID univoco della località
- `title`: Nome della località
- `lat`: Latitudine della località
- `lng`: Longitudine della località
- `istat`: Codice ISTAT del comune di appartenenza
- `city_name`: Nome del comune di appartenenza
- `province_short`: Sigla della provincia

Il DataFrame viene pulito rimuovendo i duplicati basati sulle coordinate (latitudine e longitudine) per garantire l'unicità di ogni punto geografico.

Infine, viene creata una lista di coordinate (`ors_coordinates`) nel formato `[Longitudine, Latitudine]`, pronta per essere utilizzata con servizi di routing come OpenRouteService (ORS).

Il conteggio finale dei comuni unici viene stampato, insieme a un avviso se l'estrazione non ha recuperato il numero atteso di record, suggerendo che l'API potrebbe non supportare completamente la paginazione.

In [6]:
import requests
import pandas as pd
import math
import sys

# --- Configurazione API ---
BASE_URL = "https://emiliaromagnaturismo.it/opendata/v1/localities"
# Usiamo il limite massimo che l'API accetta senza errore 400
API_LIMIT = 100
TOTAL_RECORDS = 406

# Calcolo del numero di pagine (anche se l'API potrebbe ignorare 'page')
num_pages = math.ceil(TOTAL_RECORDS / API_LIMIT)

all_localities_data = []

print("=" * 60)
print(f"FASE 1: Estrazione Località da Emilia Romagna Turismo")
print(f"Totale record teorici: {TOTAL_RECORDS}, Richieste: {num_pages} (limit={API_LIMIT})")
print("=" * 60)

# --- Estrazione Dati tramite Paginazione ---
for page in range(1, num_pages + 1):
    params = {
        'page': page,
        'limit': API_LIMIT # Parametro corretto per la dimensione della pagina
    }

    print(f"Richiesta Pagina {page}/{num_pages}...")

    try:
        response = requests.get(BASE_URL, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        localities = data.get('data', [])

        # Estrazione dei campi
        for item in localities:
            # Controllo di sicurezza per coordinate e dati di città
            if (item.get('location') and item['location'].get('lat') is not None and
                item['location'].get('lng') is not None and item['location'].get('city')):

                locality_data = {
                    'id': item.get('id'),
                    'title': item.get('title'),
                    'lat': item['location']['lat'],
                    'lng': item['location']['lng'],
                    'istat':item['location']['city'].get('istat'),
                    'city_name': item['location']['city'].get('city'),
                    'province_short': item['location']['city'].get('province_short'),
                }
                all_localities_data.append(locality_data)

    except requests.exceptions.RequestException as e:
        print(f"Errore CRITICO nella richiesta per la Pagina {page}: {e}")
        # Termina il programma se c'è un errore di rete o server
        sys.exit(1)

print("-" * 60)
print(f"Estrazione completata. Totale record grezzi estratti: {len(all_localities_data)}")

# --- Pulizia e Preparazione del DataFrame ---
df_comuni = pd.DataFrame(all_localities_data)

# Il passo di rimozione dei duplicati è CRUCIALE se l'API ignora 'page'
df_comuni = df_comuni.drop_duplicates(subset=['id', 'istat', 'title', 'lat', 'lng']).reset_index(drop=True)

final_count = len(df_comuni)
print(f"Conteggio finale dei comuni unici pronti per ORS: {final_count}")

if final_count < 400:
    print(f"\nAVVISO: L'estrazione è incompleta (solo {final_count} su 406).")
    print("Probabilmente l'API ignora il parametro 'page' e restituisce solo i primi 100 record.")
    print("La matrice delle distanze verrà calcolata solo tra questi comuni.")

# --- Creazione della lista finale di coordinate per ORS ---

# OSRM/ORS richiede le coordinate nel formato [Longitudine, Latitudine]
ors_coordinates = df_comuni[['lng', 'lat']].values.tolist()

# Salviamo il DataFrame in CSV per riferimento futuro
# df_comuni.to_csv('comuni_emilia_romagna_per_ors.csv', index=False)

print("\n" + "=" * 60)
print("Risultato: Lista 'ors_coordinates' pronta per il calcolo Matrice.")
print(f"Esempio (primi 3 punti): {ors_coordinates[:3]}")
print("=" * 60)

FASE 1: Estrazione Località da Emilia Romagna Turismo
Totale record teorici: 406, Richieste: 5 (limit=100)
Richiesta Pagina 1/5...
Richiesta Pagina 2/5...
Richiesta Pagina 3/5...
Richiesta Pagina 4/5...
Richiesta Pagina 5/5...
------------------------------------------------------------
Estrazione completata. Totale record grezzi estratti: 406
Conteggio finale dei comuni unici pronti per ORS: 406

Risultato: Lista 'ors_coordinates' pronta per il calcolo Matrice.
Esempio (primi 3 punti): [[9.593987999999968, 45.087877], [9.8698319, 45.0496283], [9.4390093, 45.0596156]]


# Script per il calcolo delle distanze con OpenRouteService

Questo script calcola la matrice delle distanze stradali tra un insieme di località utilizzando l'API di OpenRouteService (ORS).

**Prerequisiti:**

- Un token API valido per OpenRouteService. Assicurati di averlo memorizzato nelle Colab Secrets con il nome `ORS`.
- Un DataFrame Pandas chiamato `df_comuni` e una lista di coordinate `ors_coordinates`, ottenuti ad esempio eseguendo lo script di importazione delle località da Emilia Romagna Turismo.

**Funzionamento:**

1. **Configurazione ORS:** Viene inizializzato il client ORS utilizzando il token API fornito.
2. **Inizializzazione Matrice:** Viene creato un DataFrame vuoto (`distance_matrix_df`) con gli stessi nomi delle località sia per gli indici di riga che per le colonne. Questo DataFrame verrà popolato con le distanze.
3. **Calcolo Distanze:** Lo script itera su ogni località (origine). Per ciascuna origine, effettua una richiesta all'API ORS per calcolare la distanza stradale verso tutte le altre località (destinazioni). Viene utilizzato il profilo di routing 'driving-car' e richiesta la metrica di distanza in metri.
4. **Gestione Limiti API:** Viene introdotta una pausa (`SLEEP_TIME`) tra le richieste per rispettare i limiti di rate dell'API pubblica di ORS (solitamente 40 richieste al minuto). In caso di errori API (ad es. superamento del limite di richieste), viene fatta una pausa più lunga.
5. **Post-Elaborazione:** Una volta completato il calcolo, la matrice viene convertita da metri a chilometri (`distance_matrix_km`). I valori mancanti (NaN) vengono riempiti assumendo che la distanza A->B sia uguale alla distanza B->A, e la diagonale (distanza di un punto da se stesso) viene impostata a 0.
6. **Output e Salvataggio:** La matrice delle distanze finale (in Km) viene stampata in anteprima e salvata in un file CSV (`matrice_distanze_XXX_comuni.csv`, dove XXX è il numero di comuni).

**Nota:** La quota massima giornaliera dell'API gratuita di ORS è di 500 richieste. Questo script, calcolando una matrice completa per 404 comuni, consumerà circa 404 richieste, esaurendo quasi tutta la quota giornaliera. Sarà necessario attendere 24 ore per poter eseguire nuovamente lo script completo con lo stesso token API.

In [ ]:
from google.colab import userdata
import pandas as pd
import openrouteservice
import time
import sys

# --- Configurazione ORS ---
# Inserisci la tua API Key di OpenRouteService qui!
ORS_API_KEY = 'YOUR TOKEN ORS'
if ORS_API_KEY == 'YOUR TOKEN ORS' or ORS_API_KEY is None:
    print("\nERRORE: Inserisci il tuo token ORS per procedere con il calcolo delle distanze.")
    sys.exit(1)

client = openrouteservice.Client(key=ORS_API_KEY)
final_count = len(df_comuni)

# --- Inizializzazione Matrice di Risultato ---
# Usiamo i nomi dei punti (title) per etichettare righe e colonne.
final_count = len(df_comuni)
title_labels = df_comuni['title'].tolist()

distance_matrix_df = pd.DataFrame(
    index=title_labels,
    columns=title_labels,
    dtype=float
)

print("\n" + "=" * 60)
print(f"FASE 2: Inizio Calcolo Matrice ORS ({final_count}x{final_count})...")
print(f"Tempo stimato: ~{final_count * 1.6 / 60:.1f} minuti (con pausa di {1.6}s).")
print("=" * 60)

SLEEP_TIME = 2.5  # Pausa di sicurezza tra le richieste (per limite 40/min)

# --- Loop: Calcolo 1 Origine vs. Tutte le Destinazioni ---
for i in range(final_count):

    origin_name = title_labels[i]
    print(f"Richiesta {i + 1}/{final_count}: calcolo distanze da {origin_name}...")

    try:
        # Calcola la matrice 1xN (1 Origine vs. tutte le Destinazioni)
        result = client.distance_matrix(
            locations=ors_coordinates,
            sources=[i], # Indice dell'origine corrente
            destinations=list(range(final_count)), # Tutti i punti come destinazioni
            profile='driving-car',
            metrics=['distance'] # Richiedi la distanza stradale in metri
        )

        # Estrazione e salvataggio dei risultati
        # 'distances' è una lista di liste. Con 1 sorgente, prendiamo il primo elemento [0]
        distances = result['distances'][0]

        # Inseriamo i risultati nella riga 'i' (corrispondente all'origine)
        distance_matrix_df.iloc[i, :] = distances

    except Exception as e:
        print(f"\n!!! Errore ORS per {origin_name}: {e} !!!")
        distance_matrix_df.iloc[i, :] = float('nan')
        time.sleep(10) # Pausa più lunga in caso di errore API (es. Rate Limit)

    # Pausa per rispettare il rate limit dell'API pubblica
    time.sleep(SLEEP_TIME)

print("=" * 60)
print("Calcolo Matrice COMPLETATO.")

# --- FASE 3: Post-Elaborazione e Stampa ---

# 1. Converti in Km
distance_matrix_km = distance_matrix_df / 1000

# 2. Rendi la matrice simmetrica e pulisci la diagonale
# Riempi i valori mancanti (NaN) copiando la trasposta (Distanza A->B = B->A)
distance_matrix_km = distance_matrix_km.combine_first(distance_matrix_km.T).fillna(0)

# 3. Visualizzazione e Salvataggio
print("\nMatrice delle Distanze Finali (in Km - primi 5):\n")
print(distance_matrix_km.head())
print(f"\nMatrice creata, dimensione {distance_matrix_km.shape[0]}x{distance_matrix_km.shape[1]}")

# Salvataggio su file CSV
distance_matrix_km.to_csv(f'matrice_distanze_{final_count}_comuni.csv')

# Script per il calcolo delle durate con OpenRouteService

Questo script è molto simile al precedente, ma invece di calcolare le distanze stradali, calcola le **durate** stimate di viaggio tra le stesse località utilizzando l'API di OpenRouteService (ORS).

**Prerequisiti:**

- Un client ORS inizializzato (variabile `client`), ottenuto fornendo un token API valido.
- Un DataFrame Pandas chiamato `df_comuni` e una lista di coordinate `ors_coordinates`.

**Funzionamento:**

1. **Inizializzazione Matrice:** Viene creato un DataFrame vuoto (`duration_matrix_df`) per memorizzare le durate, utilizzando i nomi delle località come indici e colonne.
2. **Calcolo Durate:** Similmente allo script delle distanze, viene effettuato un loop che per ogni località di origine, richiede all'API ORS la durata di viaggio verso tutte le altre destinazioni. Viene sempre utilizzato il profilo 'driving-car', ma la metrica richiesta è 'duration'.
3. **Gestione Limiti API:** Anche qui è presente una pausa (`SLEEP_TIME`) per rispettare i limiti di rate dell'API.
4. **Post-Elaborazione:** Una volta ottenute tutte le durate (inizialmente in secondi), la matrice viene convertita in **minuti** (`duration_matrix_min`). Anche questa matrice viene resa simmetrica e la diagonale impostata a zero.
5. **Output e Salvataggio:** La matrice delle durate finale (in minuti) viene stampata in anteprima e salvata in un file CSV (`matrice_durate_XXX_comuni.csv`).

Questo script completa il set di dati necessario per analisi basate sia sulla distanza che sul tempo di viaggio tra le località.

In [ ]:
# --- Inizializzazione Matrice di Risultato ---
# La matrice memorizzerà inizialmente i valori in secondi.
duration_matrix_df = pd.DataFrame(
    index=title_labels,
    columns=title_labels,
    dtype=float
)

print("\n" + "=" * 60)
print(f"FASE 2B: Inizio Calcolo Matrice DURATE ({final_count}x{final_count})...")
print("I risultati intermedi sono in secondi.")
print("=" * 60)

SLEEP_TIME = 2.5  # Pausa di sicurezza tra le richieste

# --- Loop: Calcolo 1 Origine vs. Tutte le Destinazioni ---
for i in range(final_count):

    origin_name = title_labels[i]
    print(f"Richiesta {i + 1}/{final_count}: calcolo durate da {origin_name}...")

    try:
        # Calcola la matrice 1xN (1 Origine vs. tutte le Destinazioni)
        result = client.distance_matrix(
            locations=ors_coordinates,
            sources=[i], # Indice dell'origine corrente
            destinations=list(range(final_count)), # Tutti i punti come destinazioni
            profile='driving-car',
            metrics=['duration'] # RICHIEDI LA DURATA IN SECONDI
        )

        # Estrazione e salvataggio dei risultati
        # 'durations' è una lista di liste. Con 1 sorgente, prendiamo il primo elemento [0]
        durations = result['durations'][0]

        # Inseriamo i risultati nella riga 'i' (corrispondente all'origine)
        duration_matrix_df.iloc[i, :] = durations

    except Exception as e:
        print(f"\n!!! Errore ORS per {origin_name}: {e} !!!")
        duration_matrix_df.iloc[i, :] = float('nan')
        time.sleep(10) # Pausa più lunga in caso di errore API (es. Rate Limit)

    # Pausa per rispettare il rate limit dell'API pubblica
    time.sleep(SLEEP_TIME)

print("=" * 60)
print("Calcolo Matrice COMPLETATO.")

# --- FASE 3: Post-Elaborazione e Salvataggio (Durata) ---

# 1. Converti in Minuti
# La durata è in secondi, quindi la dividiamo per 60.
duration_matrix_min = duration_matrix_df / 60

# 2. Rendi la matrice simmetrica e pulisci la diagonale
# Riempi i valori mancanti (NaN) copiando la trasposta (Durata A->B = B->A)
duration_matrix_min = duration_matrix_min.combine_first(duration_matrix_min.T).fillna(0)

# 3. Visualizzazione e Salvataggio
print("\nMatrice delle Durate Finali (in Minuti - primi 5):\n")
print(duration_matrix_min.head())
print(f"\nMatrice creata, dimensione {duration_matrix_min.shape[0]}x{duration_matrix_min.shape[1]}")

# Salvataggio su file CSV
duration_matrix_min.to_csv(f'matrice_durate_{final_count}_comuni.csv')
print(f"File salvato: matrice_durate_{final_count}_comuni.csv")